In [99]:
import numpy as np
import pandas as pd

In [100]:
counter = 0

size = 50000000

click = np.zeros(size, np.int8)
market_price = np.zeros(size, np.int16)
features = np.zeros([24, size], np.int64)


with open('./dataset/train_set') as f:
    i = 0
    for line in f:
        raw = line.rstrip().split('\t')
        #print(raw)
        click[i] = int(raw[0])
        market_price[i] = int(raw[1])
        for idx in range(24):
            features[idx, i] = int(raw[idx+2].split(':')[0])

            
        i += 1
        if i == size:
            break
        if i % 5000000 == 0:
            print('progress', i)


progress 5000000
progress 10000000
progress 15000000
progress 20000000
progress 25000000
progress 30000000
progress 35000000
progress 40000000
progress 45000000


In [101]:
raw_df = pd.DataFrame(
    features.T,
    columns=['feature_'+str(n) for n in range(24)]
)
raw_df['click'] = click
raw_df['market_price'] = market_price

In [102]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000000 entries, 0 to 49999999
Data columns (total 26 columns):
 #   Column        Dtype
---  ------        -----
 0   feature_0     int64
 1   feature_1     int64
 2   feature_2     int64
 3   feature_3     int64
 4   feature_4     int64
 5   feature_5     int64
 6   feature_6     int64
 7   feature_7     int64
 8   feature_8     int64
 9   feature_9     int64
 10  feature_10    int64
 11  feature_11    int64
 12  feature_12    int64
 13  feature_13    int64
 14  feature_14    int64
 15  feature_15    int64
 16  feature_16    int64
 17  feature_17    int64
 18  feature_18    int64
 19  feature_19    int64
 20  feature_20    int64
 21  feature_21    int64
 22  feature_22    int64
 23  feature_23    int64
 24  click         int8 
 25  market_price  int16
dtypes: int16(1), int64(24), int8(1)
memory usage: 9.1 GB


In [103]:
raw_df.to_parquet('./raw_df.parquet')

In [104]:
def to_parquet_with_split(df, file_path_template):
    partial_size = 5000000
    size = len(df)
    idx = 0
    while True:
        start = partial_size * idx
        end = start + partial_size
        _df = df[start:end]
        file_name = file_path_template.format(idx)
        _df.to_parquet(file_name)
        print(file_name, start, end)
        idx += 1
        if end > size:
            break

In [105]:
path_template = "./to_upload/raw_{}.parquet"

In [106]:
to_parquet_with_split(raw_df, path_template)

./to_upload/raw_0.parquet 0 5000000
./to_upload/raw_1.parquet 5000000 10000000
./to_upload/raw_2.parquet 10000000 15000000
./to_upload/raw_3.parquet 15000000 20000000
./to_upload/raw_4.parquet 20000000 25000000
./to_upload/raw_5.parquet 25000000 30000000
./to_upload/raw_6.parquet 30000000 35000000
./to_upload/raw_7.parquet 35000000 40000000
./to_upload/raw_8.parquet 40000000 45000000
./to_upload/raw_9.parquet 45000000 50000000
./to_upload/raw_10.parquet 50000000 55000000


In [107]:
!ls -lha ./to_upload

total 2.0G
drwxr-xr-x 2 jupyter jupyter 4.0K Jan 14 18:38 .
drwxr-xr-x 5 jupyter jupyter 4.0K Jan 14 18:39 ..
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_0.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_1.parquet
-rw-r--r-- 1 jupyter jupyter  14K Jan 14 18:38 raw_10.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_2.parquet
-rw-r--r-- 1 jupyter jupyter 202M Jan 14 18:38 raw_3.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_4.parquet
-rw-r--r-- 1 jupyter jupyter 202M Jan 14 18:38 raw_5.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_6.parquet
-rw-r--r-- 1 jupyter jupyter 202M Jan 14 18:38 raw_7.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_8.parquet
-rw-r--r-- 1 jupyter jupyter 203M Jan 14 18:38 raw_9.parquet


In [109]:
!gsutil cp ./to_upload/*.parquet gs://xxxxxx

Copying file://./to_upload/raw_0.parquet [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://./to_upload/raw_1.parquet [Content-Type=application/octet-stream]...
Copying file://./to_upload/raw_10.parquet [Content-Type=application/octet-stream]...
Copying file://./to_upload/raw_2.parquet [Content-Type=applic

In [110]:
!bq load --source_format=PARQUET investigation_tnishibayashi.yoyi_train gs://xxxxxx/upload/*.parquet

Waiting on bqjob_r71272dc5e6da8a0d_00000177027b1dfd_1 ... (110s) Current status: DONE   
